# EfficientDet Code

Source: https://github.com/roboflow-ai/Monk_Object_Detection/tree/master/4_efficientdet

In [ ]:
#Cloning code repo
! git clone https://github.com/roboflow-ai/Monk_Object_Detection.git

In [ ]:
# Installing library requirements
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install efficientnet_pytorch
!pip install tensorboardX

In [ ]:
#Libraries required
import json
import os
import sys
import random
import cv2
from shutil import copy,rmtree
from google.colab import files
from tqdm import tqdm
!pip install gdown

# Preprocessing dataset

**Dataset**: [Aquarium Dataset](https://public.roboflow.com/object-detection/aquarium)

**Object classes**: fish, jellyfish, penguin, shark, puffin, stingray, starfish

In [ ]:
#Downloading dataset
!curl -L https://public.roboflow.com/ds/DPKAmRQ7E2?key=hqTXnMERKe > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#Preprocessing dataset to requirements of code library
!mkdir Aquarium
!mkdir Aquarium/annotations
!mkdir Aquarium/Annotations
!mkdir Aquarium/Images
%cp train/_annotations.coco.json Aquarium/annotations/instances_Images.json
%cp train/*.jpg Aquarium/Images/

# Training


In [ ]:
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");

In [ ]:
from train_detector import Detector
model = Detector();
root_dir = "./";
dataset_dir = "Aquarium";
img_dir = "./";
set_dir = "Images";
batch_size = 8;
image_size = 512;
lr = 0.0001;
num_epochs = 15;

In [ ]:
model.Train_Dataset(root_dir, dataset_dir, img_dir, set_dir, batch_size, image_size, use_gpu=True)

In [ ]:
#Training model
model.Model();
model.Set_Hyperparams(lr, val_interval=1, es_min_delta=0.0, es_patience=0)
model.Train(num_epochs, model_output_dir="trained/");

# Image Inference

In [ ]:
#Downloading pre-trained model and images
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");
!gdown https://drive.google.com/uc?id=17-pdOsShbrxypsnbiNJsmSFZ_T87Q42S
!gdown https://drive.google.com/uc?id=1U58p1TA23cW6JdA7FRiwMa5sD_fg_kwl
!gdown https://drive.google.com/uc?id=1HZfHFSErtZ8UdJzj0DI6y_Asz3u-dcWY
!gdown https://drive.google.com/uc?id=1gflRCxOn35ehtp7HQPIELqfkFz0w4wQf
model_dir = 'trained/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
copy('signatrix_efficientdet_coco.pth', model_dir)
copy('signatrix_efficientdet_coco.onnx', model_dir)

In [ ]:
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");
from infer_detector import Infer
model_inf = Infer();
model_inf.Model(model_dir="trained/")
with open('class_list.json') as json_file:
    data = json.load(json_file)
class_list = []
for category in data['categories']:
  class_list.append(category['name'])

In [ ]:
img_path = "image_1.jpeg"
duration, scores, labels, boxes = model_inf.Predict(img_path, class_list, vis_threshold=0.2);
from IPython.display import Image
display(Image(filename=img_path)) 

In [ ]:

from IPython.display import Image
Image(filename='output.jpg') 

# Video Inference

In [ ]:
#Download trained model
!gdown https://drive.google.com/uc?id=17-pdOsShbrxypsnbiNJsmSFZ_T87Q42S
!gdown https://drive.google.com/uc?id=1U58p1TA23cW6JdA7FRiwMa5sD_fg_kwl
!gdown https://drive.google.com/uc?id=1gflRCxOn35ehtp7HQPIELqfkFz0w4wQf
model_dir = 'trained/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
copy('signatrix_efficientdet_coco.pth', model_dir)
copy('signatrix_efficientdet_coco.onnx', model_dir)

In [ ]:
#Download sample videos
!gdown https://drive.google.com/uc?id=1VY_dJNIvOLuiEYS08-Z8EqxWV3V3inlC
!gdown https://drive.google.com/uc?id=1ZAcSUx8NWWZOBKu424RP5PRN1tWNrgKd
!gdown https://drive.google.com/uc?id=1X744_A08VJEI1K0y6ELDe4NVOgH8zdi0
!gdown https://drive.google.com/uc?id=10CpRyuv4Fu6r1fJIrkGEAvxT_ts9rRto

In [ ]:
#Function to perform video inference
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");
from infer_detector import Infer
def video_inf(input_video, output_video):
    inp_video_capture = cv2.VideoCapture(input_video)
    frame_width, frame_height = int(inp_video_capture.get(3)), int(inp_video_capture.get(4))
    tgt_dir = 'input_imgs/'
    if not os.path.exists(tgt_dir):
        os.makedirs(tgt_dir)

    frame_count = 1
    while inp_video_capture.isOpened():
        ret, frame = inp_video_capture.read()
        if not ret:
            break
        cv2.imwrite(tgt_dir + str(frame_count) + ".jpg", frame)
        frame_count+=1
    model_inf = Infer();
    model_inf.Model(model_dir="trained/")
    with open('class_list.json') as json_file:
        data = json.load(json_file)
    class_list = []
    for category in data['categories']:
        class_list.append(category['name'])
    imgs = [tgt_dir + str(cnt)+'.jpg' for cnt in range(1,frame_count)]
    out_tgt_dir = 'output_imgs/'
    if not os.path.exists(out_tgt_dir):
        os.makedirs(out_tgt_dir)
    for cnt in range(len(imgs)):
        duration, scores, labels, boxes = model_inf.Predict(imgs[cnt], class_list, vis_threshold=0.2);
        copy('output.jpg', out_tgt_dir + str(cnt) + '.jpg')
    out_ptr = cv2.VideoWriter(output_video,
                                cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 25,
                                (frame_width, frame_height))
    imgs = [out_tgt_dir + str(cnt)+'.jpg' for cnt in range(1,frame_count)]
    for cnt in tqdm(range(len(imgs))):
        img1 = cv2.imread(imgs[cnt]);
        out_ptr.write(img1)
    out_ptr.release()
    rmtree('./input_imgs')
    rmtree('./output_imgs')

In [ ]:
input_video = 'video_1.mp4'
output_video = 'video_1_inf.mp4'
video_inf(input_video, output_video)

In [ ]:
input_video = 'video_2.mp4'
output_video = 'video_2_inf.mp4'
video_inf(input_video, output_video)

In [ ]:
input_video = 'video_3.mp4'
output_video = 'video_3_inf.mp4'
video_inf(input_video, output_video)

In [ ]:
input_video = 'video_4.mp4'
output_video = 'video_4_inf.mp4'
video_inf(input_video, output_video)

In [ ]:
#Download inferred videos
files.download("video_1_inf.mp4")
files.download("video_2_inf.mp4")
files.download("video_3_inf.mp4")
files.download("video_4_inf.mp4")